In [34]:
#Heap stuff 
type Node{T}
    # a node data structure. It contains two arguments: a key and a value. "key" 
    # is the used to rank the node, possibly by a heap. "val" is the value of the 
    # node from the perspective of its graph.
    key::T
    val::T
end

type Heap{T}
    # a heap data structure. It contains a list of node objects. Note that elements 
    # are never deleted from the node list; rather, a pointer called "last" is used 
    # to indicate that last good value in the arrays. The heaps can be either min or 
    # max heaps, as determined by the keyword argument "kind"
    nodelist::Array{Node{T},1}
    last::T
    kind::ASCIIString
end
Heap(T, kind) = Heap{T}([],0,kind)
 
function sink(heap, i)
    # arguments:
    #     heap: a heap data structure
    #     i: an index describing the location in the heap (in the base array 
    #     itself) of the node to be sunk can be found 
    # what it does:
    #     after the base node is popped from the heap, the node in the last 
    #     position is brough into the base node, at least temporarily. The proper 
    #     position for this node can be found by "sinking" it down through the heap.
    # returns:
    #    return statements are for facilitation of recursion only
    if heap.kind == "min"
        ∘ = <=
    elseif heap.kind == "max"
        ∘ = >=
    end
    if heap.last != 0
        left = heap.nodelist[i<<1]
        right = heap.nodelist[i<<1+1]
        if left.key ∘ right.key && left.key ∘ heap.nodelist[i].key
            switch = i<<1
        elseif right.key ∘ left.key && right.key ∘ heap.nodelist[i].key
            switch = i<<1+1
        else
            return
        end
        heap.nodelist[i], heap.nodelist[switch] = heap.nodelist[switch], heap.nodelist[i]
        return sink(heap, switch)
    end
end

function float(heap, i)
    # arguments:
    #     heap: a heap data structure
    #     i: an index describing the location in the heap (in the base array 
    #     itself) of the node to be floated can be found 
    # what it does:
    #     a new node is pushed onto the heap in the last position. This may not be 
    #     its rightful location in the heap. The proper position for this node can 
    #     be found by "floating" it up through the heap.
    # returns:
    #    return statements are for facilitation of recursion only
    if heap.kind == "min"
        ∘ = <
    elseif heap.kind == "max"
        ∘ = >
    end
    if i == 1
        return
    else
        if heap.nodelist[i].key ∘ heap.nodelist[i>>1].key
            heap.nodelist[i], heap.nodelist[i>>1] = heap.nodelist[i>>1], heap.nodelist[i]
            return float(heap, i>>1)
        end
    end
end

function extremum(heap)
     if heap.kind == "min"
        if typeof(heap) == Heap{UInt32}
            extreme = Node{UInt32}(2^32-1, 0)
        elseif typeof(heap) == Heap{Int64}
            extreme = Node{Int64}(2^32-1, 0)
        end
     elseif heap.kind == "max"
        if typeof(heap) == Heap{UInt32}
            extreme = Node{UInt32}(0, 0)
        elseif typeof(heap) == Heap{Int64}
            extreme = Node{Int64}(-2^32+1, 0)
        end
        return extreme
    end
end
        
function pop(heap)
    # arguments:
    #     heap: a heap data structure
    # what it does:
    #     removes the base node from the heap. Note that elements are never 
    #     deleted from the heap's lists, rather the last pointer is decremented
    # returns:
    #    the contents of the base node: its key and value
    outer = extremum(heap)
    if heap.last > 0
        returnable = heap.nodelist[1]
        heap.nodelist[1], heap.nodelist[heap.last] = heap.nodelist[heap.last], outer
        heap.last -= 1
        sink(heap, 1)
        return returnable
    else
        throw(error("Ain't no thing in the heap"))
    end
end

function push(heap, nuno)
    # arguments:
    #     heap: a heap data structure
    #     key: the quantity of the new node around which the heap is organized
    #     value: other quantity associated with the node but unimportant to the heap
    # what it does:
    #     adds a new node to the heap. Note that if the heap's lists are too short to 
    #     contain the new nodes children, dummy elements are added to the lists. The 
    #     keys of these dummies are the maximum allowed by the data type if a min heap, 
    #     zero if a max heap. This is done to facilitate sinking after pop.
    # returns:
    #    no returns
    outer = extremum(heap)
    if size(heap.nodelist)[1] == 0
        push!(heap.nodelist, nuno)
        heap.last = 1
    else
        heap.last += 1
        heap.nodelist[heap.last] = nuno
    end
    if 2*heap.last+1 > size(heap.nodelist)[1]
        push!(heap.nodelist,outer, outer)
    end
    float(heap, heap.last)
end

push (generic function with 1 method)

In [48]:
function explore(graph, heap, shortestpaths, explored)
    # arguments:
    #     graph: an adjacency matrix of an undirected graph of the form 
    #     Dict{Any,Dict{Any,Number}}.
    #     heap: a heap data structure with graph nodes as elements
    #     shortestpaths: a (partial) list of nodes in increasomg order of 
    #     shortest path
    #     explored: a list of booleans corresponding to the visitation status of 
    #     the node corresponding to the index of the array
    # what it does:
    #     Pops the top off the heap and adds a new nodes to the frontier--all 
    #     nodes which are adjacent to the node at the top of the heap and that 
    #     have not been previously explored. Nodes are not considered explored until 
    #     they have been popped off the heap. 
    # returns:
    #    the argument heap with new nodes to be explored and the base node popped off
    #    an argument shortestpaths list updated with the shortest remaining path 
    #    from the heap
    #    the argument explored list with the most recently explored node included
    current = pop(heap)
    if !explored[current.val]
        push!(shortestpaths, current)
        pathlength = current.key
        explored[current.val] = true
        for k in keys(graph[current.val])
            if !explored[k]
                push(heap, Node(pathlength + graph[current.val][k], k))
            end
        end
    end
    return heap, shortestpaths, explored
end

function Dijkstra(graph, startnode)
    # arguments:
    #     graph: an undirected adjacency list of the form Dict{Any,Dict{Any,Number}}.
    #     startnode: the starting point from which all shortest paths will be measured
    # what it does:
    #     finds the shortest path to all points from the startnode
    # returns:
    #    a complete list of nodes sorted in order of node number. Each node ion the 
    #    list contains the shortest path as its "key", and the node number as its "val"
    shortestpaths = []
    explored = [false for i = 1:maximum(keys(graph))]
    heap = Heap(Int64,"min")
    push(heap, Node{Int64}(0, startnode))
    while !all(explored)
        heap, shortestpaths, explored = explore(graph, heap, shortestpaths, explored)
    end
    return sort(shortestpaths, by=x->x.val)
end

Dijkstra (generic function with 2 methods)